In [1]:
import re
import time
import random
import requests
import urllib.request
import pandas as pd
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup
from headers import AllHeader

head = AllHeader()
link_metro = "https://www.hmetro.com.my/mutakhir/2022/11/898686/pru-15-senarai-calon-kerusi-parlimen-dun-pn"

link_harakah = (
    "https://harakahdaily.net/index.php/2018/04/29/senarai-calon-bertanding-pru14/"
)
link_e_central = "https://ecentral.my/prn-2023/"

In [21]:
def request_text(link, use_random_header=False) -> str:
    """
    Given link, return html text extracted from website

    :param link:str
    :param use_random_header:Boolear = False

    >>> link_harakah = "https://harakahdaily.net/index.php/2018/04/29/senarai-calon-bertanding-pru14/"
    >>> assert type(request_text(link_harakah)) == str, "Not return string"

    """
    if use_random_header:
        random_header = random.choice(head.headers_list)
    else:
        random_header = head.headers_list[0]

    robust_header = head.robust_header
    header_list = [{"User-Agent": random_header}, robust_header]

    response = requests.get(link, headers=random.choice(header_list))
    html_text = response.text

    assert type(html_text) == str, "request_text not return a string"
    print({"STATUS": response.status_code, "HEADER": random_header})
    # print("TEXT:\n", html_text)
    return html_text


def bs4_soup(html_text):
    soup_ = BeautifulSoup(html_text, "lxml")
    return soup_

## Harakah scraper

Pautan: [Senarai Calon Bertanding PRU14]("https://harakahdaily.net/index.php/2018/04/29/senarai-calon-bertanding-pru14/")

In [8]:
html_text = request_text(link=link_harakah, use_random_header=True)
soup_ = bs4_soup(html_text=html_text)
div_tag = soup_.find_all("div", {"class": "td-pb-span8 td-main-content"})

candidates_list = list()
location = list()
candidate = list()
for tag in div_tag:
    parents_tag = tag.find_all("div", {"class": "td-post-content tagdiv-type"})
    for p_tag_soup in parents_tag:
        p_tag = p_tag_soup.find_all("p")
        for tag in p_tag:
            # print(tag.find)
            candidates_list.append(tag)
            strong_tag = tag.find("strong").text
            location.append(strong_tag)
            em_tag = tag.find("em")
            if em_tag is not None:
                em_text = em_tag.text
                candidate.append(em_text)
            else:
                candidate.append(em_tag)


# for idx, _ in enumerate(candidates_list):
#     print(idx, _.strip())
print(location)
print(len(location))
print(candidate)
print(len(candidate))

{'STATUS': 200, 'HEADER': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.83 Safari/537.36'}
['PERLIS', 'P.001 PADANG BESAR', 'P.002 KANGAR', 'P.003 ARAU \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0', 'KEDAH', 'P.004 LANGKAWI ', 'P.005 JERLUN', 'P.006 KUBANG PASU', 'P.007 PADANG TERAP', 'P.008 POKOK SENA', 'P.009 ALOR SETAR', 'P.010 KUALA KEDAH', 'P.011 PENDAN\xa0', 'P.012 JERAI', 'P.013 SIK \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0 \xa0', 'P.014 MERBOK', 'P.015 SUNGAI PETANI', 'P.016 BALING \xa0 \xa0 \xa0 \xa0', 'P.017 PADANG SERAI', 'P.018 KULIM BANDAR BAHARU', 'KELANTAN', 'P.019 TUMPAT \xa0 \xa0 \xa0', 'P.020 PENGKALAN CHEPA', 'P.021 KOTA BHARU', 'P.022 PASIR MAS', 'P.023 RANTAU PANJANG', 'P.024 KUBANG KERIAN', 'P.025 BACHOK', 'P.026 KETEREH', 'P.027 TANAH MERAH', 'P.028 PASIR PUTEH', 'P.029 MACHANG', 'P.030 JELI', 'P.031 KUALA KRAI', 'P.032 GUA MUSANG', 'TERENGGANU', 'P.033 BESUT \xa0 \xa0 \xa0 \xa0 ', 'P.034 SETIU', 'P.035 KUALA NE

In [87]:
df = pd.DataFrame({"Location": location, "Candidates": candidate})
df

,Location,Candidates
0,PERLIS,None
1,P.001 PADANG BESAR,"Ustaz Mokhtar Senik (GS), Izizam Ibrahim (PH),..."
2,P.002 KANGAR,"Ustaz Mohamad Zahid Ibrahim (GS), Noor Amin Ah..."
3,P.003 ARAU,"Ustaz Haji Hashim Jasin (GS), Dato’ Dr Haji Ab..."
4,KEDAH,None
...,...,...
233,P.218 SIBUTI,"Ustaz Zulaihi Bakar (GS), Jemat Panjang (PH),..."
234,P.219 MIRI,"Dr Michael Teo Yu Keng (PH), Sebastian Ting Ch..."
235,P.220 BARAM,"Roland Egan (PH), Anyi Ngau (BN)"
236,P.221 LIMBANG,"Dr Ricardo Yampil Baba (PH), Hasbi Habibollah..."


# e-Central

In [25]:
html_text = request_text(link=link_e_central, use_random_header=True)
soup_ = bs4_soup(html_text=html_text)

{'STATUS': 200, 'HEADER': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/99.0.4844.83 Safari/537.36'}


In [222]:
print(html_text)

<!doctype html>
<html lang="en-US">
<head><meta charset="UTF-8"><script>if(navigator.userAgent.match(/MSIE|Internet Explorer/i)||navigator.userAgent.match(/Trident\/7\..*?rv:11/i)){var href=document.location.href;if(!href.match(/[?&]nowprocket/)){if(href.indexOf("?")==-1){if(href.indexOf("#")==-1){document.location.href=href+"?nowprocket=1"}else{document.location.href=href.replace("#","?nowprocket=1#")}}else{if(href.indexOf("#")==-1){document.location.href=href+"&nowprocket=1"}else{document.location.href=href.replace("#","&nowprocket=1#")}}}}</script><script>class RocketLazyLoadScripts{constructor(){this.v="1.2.3",this.triggerEvents=["keydown","mousedown","mousemove","touchmove","touchstart","touchend","wheel"],this.userEventHandler=this._triggerListener.bind(this),this.touchStartHandler=this._onTouchStart.bind(this),this.touchMoveHandler=this._onTouchMove.bind(this),this.touchEndHandler=this._onTouchEnd.bind(this),this.clickHandler=this._onClick.bind(this),this.interceptedClicks=[],wi

In [268]:
from itertools import zip_longest

div_tag = soup_.find("div", {"class": "ct-container-full"})
p_tag = div_tag.find_all("p", string=re.compile(r"^DUN"))
li_tag = div_tag.find_all("li", string=re.compile(r"^N\d{2}.*|^N.\d{2}.*"))
ul_tag = div_tag.find_all("ul")

p_li_tag = div_tag.find_all(["p", "li"])

candidate = dict()
# for i in p_tag:
#     print(i.text)

# for i in li_tag:
#     print(i.text)

# for i in p_li_tag:
#     print(i)

content = [x for x in p_li_tag]
content_ = content[39:-14]

for item in content_:
    item = str(item)
    if item.startswith("<p>"):
        item = (
            str(item)
            .replace("<p>", "")
            .replace("</p>", "")
            .replace("<strong>", "")
            .replace("</strong>", "")
            .strip()
        )
        key = item
        candidate[key] = []
    else:
        pass
    if item.startswith("<li>"):
        item = (
            str(item).replace("<li>", "").replace("</li>", "").replace("\\", "").strip()
        )

        item = re.sub("\\\\u", "", item)

        value = item
        candidate[key].append(value)
    else:
        pass

print(candidate)

{'DUN Kedah': ['N01 Ayer Hangat – Hisham Suhaili Osman', 'N04 Ayer Hitam – Hayazi Azizan', 'N05 Bukit kayu hitam – Zainul Abidin Mohamad', 'N08 Pedu – Datuk Seri Mahdzir Khalid', 'N09 Bukit lada – Datuk Said Ali Syed Rastan', 'N15 Anak Bukit – Nor Hasita Md Isa', 'N19 Sungai Tiang – Mohammad Fadzil Zolkifpli', 'N21 Guar Cempedak – Abdul Paris Abdul Hamid', 'N24 Jeneri – Muhamad Khizri Abu Kassim', 'N27 Pantai Merdeka – Wan Mohalina Wan Mohamad', 'N30 Bayu – Ishak Mat', 'N32 Kuala Ketil – Suriati Che Mid', 'N33 Merbau Pulas – Asmadi Abu Talib', 'N36 Bandar Baharu – Nuraini Yusoff'], 'DUN Kelantan': ['N01 Pengkalan Kubor – Zulkifli Abdullah', 'N02 Kelaboran – Mohd Rosdi Razali', 'N04 Wakaf Bharu – Abdul Mannan Md Said', 'N05 Kijang – Haris Hussin', 'N06 Chempaka – Nik Normi Nik Ayub', 'N10 Bunut Payong – Datuk Mohamed Hasnan Che Hussin', 'N11 Tendong – Noor Hariri Mohamed Noor', 'N13 Meranti – Zahri Omar', 'N15 Gual Periok – Anuar Mohamad', 'N16 Apam Putra – Datuk Akbar Salim', 'N17 Salo

In [271]:
import json

json_candidate = json.dumps(candidate, indent=4)
# print(json_candidate)

with open("candidate.json", "w", encoding="utf-8") as f:
    json.dump(json_candidate, f, ensure_ascii=False, indent=4)

In [273]:
with open("candidate.json", "r") as f:
    print(json.load(f))

{
    "DUN Kedah": [
        "N01 Ayer Hangat \u2013 Hisham Suhaili Osman",
        "N04 Ayer Hitam \u2013 Hayazi Azizan",
        "N05 Bukit kayu hitam \u2013 Zainul Abidin Mohamad",
        "N08 Pedu \u2013 Datuk Seri Mahdzir Khalid",
        "N09 Bukit lada \u2013 Datuk Said Ali Syed Rastan",
        "N15 Anak Bukit \u2013 Nor Hasita Md Isa",
        "N19 Sungai Tiang \u2013 Mohammad Fadzil Zolkifpli",
        "N21 Guar Cempedak \u2013 Abdul Paris Abdul Hamid",
        "N24 Jeneri \u2013 Muhamad Khizri Abu Kassim",
        "N27 Pantai Merdeka \u2013 Wan Mohalina Wan Mohamad",
        "N30 Bayu \u2013 Ishak Mat",
        "N32 Kuala Ketil \u2013 Suriati Che Mid",
        "N33 Merbau Pulas \u2013 Asmadi Abu Talib",
        "N36 Bandar Baharu \u2013 Nuraini Yusoff"
    ],
    "DUN Kelantan": [
        "N01 Pengkalan Kubor \u2013 Zulkifli Abdullah",
        "N02 Kelaboran \u2013 Mohd Rosdi Razali",
        "N04 Wakaf Bharu \u2013 Abdul Mannan Md Said",
        "N05 Kijang \u2013 Haris Hu

In [90]:
div_tag = soup_.find("div", {"class": "ct-container-full"})


dun_name = list()
# Find DUN Name
p_tag = div_tag.find_all("p")
for tag in p_tag:
    print("DEBUG", tag)
    if tag.find("strong"):
        if "DUN" in tag.text:
            dun_name.append(tag.text)

# Find calon name
li_tag = div_tag.find_all("li", string=re.compile(r"^N\d{2}.*|^N.\d{2}.*"))
for tag in li_tag:
    print(tag.text)

# print(div_tag)

DEBUG <p><strong>PRN 2023 : Senarai Calon PRN 6 Negeri &amp; Tarikh Mengundi</strong></p>
DEBUG <p>Tentu ramai yang tertanya-tanya, bilakah tarikh PRN 2023 akan diadakan dan mungkin sesetengah daripada rakyat Malaysia keliru mengapakah pilihan raya diadakan lagi bagi 6 negeri yang terpilih.</p>
DEBUG <p>Berikut dikongsikan lebih lanjut tentang Pilihan Raya Negeri dan calon PRN di 6 buah negeri bagi tahun 2023.</p>
DEBUG <p>Pada November 2022 yang lalu, Pilihan Raya Umum Ke-15 (PRU15) telah diadakan. </p>
DEBUG <p>Ketika PRU15 tersebut, negeri-negeri Perlis, Pahang, Perak, Sabah, Sarawak dan Melaka telah mengikut kerajaan pusat untuk membubarkan Parlimen dan DUN serentak.</p>
DEBUG <p>Walau bagaimanapun, 6 negeri lain tidak bersetuju untuk membubarkan DUN kerana bimbang jika berlakunya kejadian banjir.</p>
DEBUG <p>Untuk makluman, PRN dan PRU boleh dilakukan secara serentak kerana ia lebih menjimatkan wang negara. Dalam konteks kewangan, satu-satu pilihan raya memerlukan kos yang besar.

## Backup plan
- Copy and Paste
- Split text 
- Put in table

In [37]:
name_list = """
N01 Titi Tinggi

1. Teh Chai Aan (BN)

2. Teh Seng Chuan (PH)

3. Izizam Ibrahim (PN)

4. Khaw Hock Kong (WARISAN)

5. Mohd Shahril Md Sarif (PEJUANG)

6. Zahidi Zainul Abidin (Bebas)

Jumlah pengundi PRU15: 10,111

PRU14 Penyandang: Teh Chai Aan (BN)

Majoriti: 142

N02 Beseri

1. Ruzaini Rais (BN)

2. Mat Safar Saad (PH)

3. Haziq Asyraf Dun (PN)

4. Mohd Shamim M Nordin (WARISAN)

Jumlah pengundi PRU15: 8,914

PRU14: Penyandang: Ruzaini Rais (BN)

Majoriti: 416

N03 Chuping

1. Asmaiza Ahmad (BN)

2. Natthavuth Seng (PH)

3. Saad Seman (PN)

Jumlah pengundi PRU15: 10,542

PRU14 Penyandang: Asmaiza Ahmad (BN)

Majoriti: 1,367

N04 Mata Ayer

1. Wan Badariyah Wan Saad (PN)

2. Amran Kamarudin (PH)

3. Siti Berenee Yahya (BN)

Jumlah Pengundi PRU15: 7,617

PRU14: Penyandang: Siti Berenee Yahya (BN)

Majoriti: 720

N05 Santan

1. Che Mazlina Che Yob (PH)

2. Azmir Azizan (PN)

3. Azizan Sulaiman (BN)

Jumlah penngundi PRU15: 8,912

PRU14: Penyandang Azizan Sulaiman (BN)

Majoriti: 949

N06 Bintong

1. Syazwan Suban (PEJUANG)

2. Azlan Man (BN)

3. Azhari Ahmad (PH)

4. Hashim Suboh (Bebas)

5. Mohamad Khair Mohd Noor (WARISAN)

6. Fakhrul Anwar Ismail (PN)

Jumlah pengundi PRU15:12,136

PRU14 Penyandang: Azlan Man (BN)

Majoriti: 847

N07 Sena

1. Saiful Daniel Mohd Yusof (BN)

2. Asrul Nizan Abd Jalil (PH)

3. Marzita Mansor (PN)

4. Mohd Faizal Yunus (PEJUANG)

Jumlah pengundi PRU15: 12,303

PRU14: Penyandang Asrul Nizan Abd Jalil (PH)

Majoriti: 314

N08 Indera Kayangan

1. Gan Ay Ling (PH)

2. Lim Weng Khee (BN)

3. Pramoot Puan (PN)

4. Atan Jasin (WARISAN)

Jumlah pengundi PRU15: 11,018

PRU14 Penyandang: Gan Ay Ling (PH)

Majoriti: 3,177

N09 Kuala Perlis

1. Azahar Ahmad (Bebas)

2. Abu Bakar Hamzah (PN)

3. Muhamad Hairi Noordin (WARISAN)

4. Kamarudin Malek (BN)

5. Nor Azam Karap (PH)

Jumlah pengundi PRU15: 10,654

PRU14: Penyandang Nor Azam Karap (PH)

Majoriti: 1,254

N10 Kayang

1. Asrul Aimran Abdul Jalil (PN)

2. Hamizan Hassan (BN)

3. Khairuddin Abdullah (WARISAN)

4. Wan Kharizal Wan Khazim (PH)

Jumlah pengundi PRU15: 9,827

PRU14: Penyandang: Hamizan Hassan (BN)

Majoriti: 435

N11 Pauh

1. Megat Hashirat Hassan (PN)

2. Azhar Ameir (PH)

3. Syed Atif Syed Abu Bakar (BN)

4. Mohd Khalid Ahmad (WARISAN)

Jumlah pengundi PRU15: 11,200

PRU14: Penyandang Rozieana Ahmad (BN)

Majoriti: 143

N12 Tambun Tulang

1. Wan Zikri Afthar Ishak (PN)

2. Ismail Kassim (BN)

3. Muhammad Syahmi Suhaimi (PH)

4. Maton Din (PEJUANG)

Jumlah pengundi PRU15: 10,535

PRU14: Penyandang Ismail Kassim (BN)

Majoriti: 1,180

N13 Guar Sanji

1. Hasparizal Hassan (PH)

2. Redzuan Hashim (PN)

3. Abdul Malik Abdullah (PEJUANG)

4. Roziena Ahmad (BN)

Jumlah pengundi PRU15: 9,233

PRU14: Penyandang Redzuan Hashim (PN)

Majoriti: 328

N14 Simpang Empat

1. Nurul Hisham Yaakob (BN)

2. Amran Darus (PH)

3. Izhar Kamarudin Sudin (PEJUANG)

4. Dr Wan Noralhakim Shaghir Saad (WARISAN)

5. Razali Saad (PN)

6. Ammar Hassan (Bebas)

Jumlah pengundi PRU15: 8,547

PRU14: Penyandang Nurul Hisham Yaakob (BN)

Majoriti: 82

N15 Sanglang

1. Zainuddin Yom (WARISAN)

2. Dr Ahmad Fadzhil Mohamad (PH)

3. Sharipudin Ahmad (BN)

4. Mohd Shukri Ramli (PN)

Jumlah pengundi PRU15: 8,672

PRU14: Penyandang Mohd Shukri Ramli (PN)

Majoriti: 42

PERAK

N1 PENGKALAN HULU

Jumlah Pengundi: 20,157

1. Mohamad Amin Roslan (PN-PAS)

2. Aznel Ibrahim (BN-UMNO)

3. Abg Man (PH)

PRU14:AZNEL IBRAHIM (BN)

Jumlah Undi: 5,716

Majoriti: 2,375

N2 TEMENGOR

Jumlah Pengundi: 27,408

1. Salbiah Mohamed (BN-UMNO)

2. Mohd Noor Abdul Rahman (PN-PAS)

3. Ahmad Safwan Mohamad (PH-PKR)

PRU14:SALBIAH MOHAMED (BN)

Jumlah Undi: 7,823

Majoriti: 3,935

N3 KENERING

Jumlah Pengundi: 21,596

1. Rohaida Mohamad Yaakun (BN-UMNO)

2. Husairi Ariffin (PN-PAS)

3. Mohd Jamean Zulkifli (PH-PKR)

PRU14:TARMIZI IDRIS (BN)

Jumlah Undi: 7,379

Majoriti: 3,268

N4 KOTA TAMPAN

Jumlah Pengundi: 15,354

1. Saarani Mohd (BN-UMNO)

2. Mohd Jamil Yahya (PN-PAS)

3. Mohd Sabri Abdul Manaf (PH)

PRU14:SAARANI MOHAMAD (BN)

Jumlah Undi: 5,183

Majoriti: 2,302

N5 Selama

Jumlah Pengundi: 22,339

1. Faizul Rizal Zulkifli (BN-UMNO)

2. Mohd Akmal Kamarudin (PN-PAS)

3. Razali Ismail (PH)

4. Osman Sidek (Bebas)

PRU14:AKMAL KAMARUDIN (PAS)

Jumlah Undi: 5,516

Majoriti: 289

N6 KUBU GAJAH

Jumlah Pengundi: 21,358

1. Osman Ahmad (BN-UMNO)

2. Khalil Yahaya (PN-PAS)

3. Rusli Bakar ( PH)

4. Fuaddin Kamaruddin (PEJUANG)

PRU14: KHALIL YAHAYA (PAS)

Jumlah Undi: 5,786

Majoriti: 180

N7 BATU KURAU

Jumlah Pengundi: 22,022

1. Mohd Najmuddin Elias Al-Hafiz (PN-BERSATU)

2. Saliza Ahmad (BN-UMNO)

3 Muhamad Aiman ​​Azizuddi Md Husin (PH-PKR)

4.Zulkifli (PEJUANG)

PRU14:DATUK DR MUHAMMAD AMIN ZAKARIA (BN)

Jumlah Undi: 6,168

Majoriti: 2,308

N8 TITI SERONG

Jumlah Pengundi: 35,900

1. Hakimi Hamzi Hayat (PN-PAS)

2. Shahrul Nizam Razali (BN-UMNO)

3. Muhamad Nakhaie Abd Wahab ( PH)

PRU14: HASNUL ZULKARNAIN (PKR)

Jumlah Undi: 7,600

Majoriti: 140

N9 KUALA KURAU

Jumlah Pengundi: 32,602

1. Abdul Yunus Jamahri (PN-BERSATU)

2. Mohd Jamil Jahaya (BN-UMNO)

3. Anuar (PH)

PRU14: YUNUS JAMHARI (PKR)

Jumlah Undi: 8,655

Majoriti: 2,643

N10 ALOR PONGSU

Jumlah Pengundi: 28,656

1. Noor Azman Ghazali (PN)

2. Sham Mat Sahat (BN-UMNO)

3. Khairul Azman Ahmad (PH-PKR)

PRU14: DATUK SHAM MAT SAHAT (BN)

Jumlah Undi: 6,556

Majoriti: 1,162

N11 GUNONG SEMANGGOL

Jumlah Pengundi: 27,396

1. Razman Zakaria (PN-PAS)

2. Nazirul Jamaluddin (BN-UMNO)

3. Sheikh Khuzaifah (PH)

PRU14:RAZMAN ZAKARIA (PAS)

Jumlah Undi: 7,444

Majoriti: 1,903

N12 SELINSING

Jumlah Pengundi: 24,241

1. Salehuddin (PN)

2. Mohamad Noor Dawoo (BN-UMNO)

3. Saqif Ansorullah (PH)

PRU14: MOHAMAD NOOR DAWOO (BN)

Jumlah Undi: 5,167

Majoriti: 97

N13 KUALA SEPETANG

Jumlah Pengundi: 94,253

1. Ahmad Man (PN-BERSATU)

2. Mohd Kamaruddin Abu Bakar (BN-UMNO)

3. Zainal Azman Abu Seman (PH-PKR)

PRU14: MOHD KAMARUDDIN ABU BAKAR (BN)

Jumlah Undi: 8,993

Majoriti: 329

N14 CHANGKAT JERING

Jumlah Pengundi: 37,003

1. Rahim Ismail (PN-PAS)

2. Ahmad Saidi Mohamad Daud (BN-UMNO)

3. Megat Shariffudin (PH)

4. Nazar Talib (PEJUANG)

PRU14: AHMAD SAIDI MOHAMAD DAUD (BN)

Jumlah Undi: 8,818

Majoriti: 1,922

N15 TRONG

Jumlah Pengundi: 17,028

1. Faisal Abdul Rahman (PN-PAS)

2. Jamilah Zakaria (BN-UMNO)

3. Junaida Jamaluddin (PH-PKR)

PRU14:JAMILAH ZAKARIA (BN)

Jumlah Undi: 5,139

Majoriti: 1,898

N16 KAMUNTING

Jumlah Pengundi: 39,850

1. Mohd Fakhruddin Abdul Aziz (PN-PAS)

2. Ahmad Sahalimin Ahmad Shaffie (BN-UMNO)

3. Dr Fadhil (PH)

PRU14: DR MUHD FADHIL NURUDDIN (PKR)

Jumlah Undi: 9,898

Majoriti: 1,780

N17 POKOK ASSAM

Jumlah Pengundi: 36,513

1. Shariful Juhari Abu Kassim (PN-BERSATU)

2. Ang Sui Eng (BN-MCA)

3. Ong Seng Guan (PH)

4. Mohd Yusoff Abdul Hamid (Bebas)

5. Leow Thye Yin (Bebas)

PRU14: LEOW THYE YIH (PKR)

Jumlah Undi: 16,954

Majoriti: 12,032

N18 AULONG

Jumlah Pengundi: 45,203

1. Koay Boon Hui (PN-GERAKAN)

2. Teh Kok Lim (PH)

3. Shanmuganelloo (BN)

4. Lee Seng Yap (Bebas)

5. Abu Bakar Abdul Rahman (Bebas)

PRU14: NGA KOR MING (PKR)

Jumlah Undi: 18,123

Majoriti: 12,064

N19 CHENDEROH

Jumlah Pengundi: 17,782

1. Syed Lukman Hakim Syed Mohd Zin (PN-BERSATU)

2. Khairuddin Mohamed Azahari (BN-UMNO)

3. Mohd Khairol Irwan Mat Khairi (PH-PKR)

4. Ahmad Rodzi Abdul Rahim (PEJUANG)

PRU14:ZAINUN MAT NOOR (BN)

Jumlah Undi: 5,546

Majoriti: 2,370

N20 LUBOK MERBAU

Jumlah Pengundi: 20,904

1. Azizi Mohamed Ridzuan (PN-PAS)

2. Mohamad Azir Ismail (BN-UMNO)

3. Zaiton (PH)

PRU14:DR JURIJ JALALUDIN (BN)

Jumlah Undi: 4,908

Majoriti: 409

N21 LINTANG

Jumlah Pengundi: 33,107

1. Ahmad Daslie Osman (PN-BERSATU)

2. Mohd Zolkafly Harun (BN-UMNO)

3. Zaim Sidqi Zulkifly (PH-PKR)

PRU14:DATUK MOHD ZOLKAFLY HARUN (BN)

Jumlah Undi: 10,605

Majoriti: 4,582

N22 JALONG

Jumlah Pengundi: 39,288

1. Naran Singh Asa Singh (PN-GERAKAN)

2. Pan Chean Chang (BN-MCA)

3. Loh Sze Yee (PH)

4. Indrani (Bebas)

PRU14: LOH SZE YEE (PKR)

Jumlah Undi: 16,138

Majoriti: 9,602

N23 MANJOI

Jumlah Pengundi: 160,558

1. Asmuni Awi (PH)

2. Datuk Azizul Kama Abd Aziz (BN-UMNO)

3. Hafez Sabri (PN-PAS)

PRU14: ASMUNI AWI (PKR)

Jumlah Undi: 20,052

Majoriti: 3,532

N24 HULU KINTA

Jumlah Pengundi: 74,292

1. Muhamad Arafat Varisai Mahamad (PH-PKR)

2. Mazlan Abd Rahman (BN-UMNO)

3. Puteri Holijah Muhamad Rali (PN-BERSATU)

4. S Murugiah (Bebas)

PRU14: MUHAMAD ARAFAT (PKR)

Jumlah Undi: 17,766

Majoriti: 3,713

N25 CANNING

Jumlah Pengundi: 41,021

1. Pang Boon Yang (PN-GERAKAN)

2. Woo Kok Toong (BN-MCA)

3. Jenny Choy Tsi Jen (PH)

PRU14: JENNY CHOY TSI JEN (PKR)

Jumlah Undi: 21,268

Majoriti: 18,292

N26 TEBING TINGGI

Jumlah Pengundi: 37,223

1. Mohammad Iskandar Abdul Rahman (PN-BERSATU)

2. Khoo Boon Chuan (BN-MCA)

3. Aziz Bari (PH)

4. Mior Haidir (PEJUANG)

PRU14: DR AZIZ BARI (PKR)

Jumlah Undi: 10,334

Majoriti: 3,927

N27 PASIR PINJI

Jumlah Pengundi: 39,934

1. Soo Poh Yeow (BN-MCA)

2. Lam Kin Yip (PN-GERAKAN)

3. Goh See Hua (PH)

PRU14:HOWARD LEE (PKR)

Jumlah Undi: 23,282

Majoriti: 20,856

N28 BERCHAM

Jumlah Pengundi: 52,750

1. Lim Jin Sheng (PN-GERAKAN)

2. Ho Wai Mun (BN-MCA)

3. Ong Boon Piow (PH)

PRU14: ONG BOON PIOW (PKR)

Jumlah Undi: 24,647

Majoriti: 20,709

N29 KEPAYANG

Jumlah Pengundi: 52,750

1. Dr Richard Ng (PN-BERSATU)

2. Lim Huey Shan (BN-MCA)

3. Nga Kor Ming (PH)

PRU14: DR KO CHUNG SEN (PKR)

Jumlah Undi: 12,417

Majoriti: 9,495

N30 BUNTONG

Jumlah Pengundi: 36,714

1. Thulsi Manogaran (PH)

2. Sivasubramaniam Athinarayanan (PN-BERSATU)

3. S Jayagopi (BN)

4. Muhammad Faiz Abdullah (Bebas)

5. Sebastiar (Bebas)

PRU14: SIVASUBRAMANIAM (PKR)

Jumlah Undi: 18,005

Majoriti: 15,323

N31 JELAPANG

Jumlah Pengundi: 35,088

1. Hamidah Muhamad (BN)

2. Cheah Pou Hian (PH)

3. Navinten Sundarajan (PN)

PRU14: CHEAH POU HIAN (PKR)

Jumlah Undi: 16,940

Majoriti: 15,187

N32 MENGLEMBU

Jumlah Pengundi: 38,689

1. Wong Kean Rong (PN-BERSATU)

2. Chaw Kam Foon (PH)

3. Phoon Kai Mun (BN-MCA)

PRU14: CHAW KAM FOON (PKR)

Jumlah Undi: 20,456

Majoriti: 17,948

N33 TRONOH

Jumlah Pengundi: 38,119

1. Leong Chee Wai (BN-MCA)

2. Sin Koon Yen (PN-GERAKAN)

3. Steven Tiw (PH)

4. Roslan Bin Ismail (PEJUANG)

5. Bryan Yong Wu Sean (Bebas)

6. Meor Shahimudin Hasim (Bebas)

PRU14:YONG CHOO KIONG (PKR)

Jumlah Undi: 15,061

Majoriti: 10,501

N34 BUKIT CHANDAN

Jumlah Pengundi: 22,194

1. Azizul Rahim Abd Rahman (BN-UMNO)

2. Dr Hashim Bujang (PN-BERSATU)

3. Mohamad Khairul Amir Sabri (PH-PKR)

PRU14: MASLIN SHAM RAZMAN (BN)

Jumlah Undi: 5,929

Majoriti: 464

N35 MANONG

Jumlah Pengundi: 24,791

1. Mustafa Shah Abdul Hamid (BN-UMNO)

2. Burhanuddin Ahmad (PN-PAS)

3. Jamsari Mahamood (PH)

PRU14: MOHAMAD ZURAIMI RAZALI (BN)

Jumlah Undi: 6,267

Majoriti: 571

P68 BERUAS

Jumlah Pengundi: 108,249

1. Ding Siew Chee (BN-MCA)

2. Ong Kean Sing (PN-GERAKAN)

3. Ngeh Koo Ham (PH)

PRU14:DATUK NGEH KOO HAM (PKR)

Jumlah Undi: 41,231

Majoriti : 27,954

N36 PENGKALAN BAHARU

Jumlah Pengundi: 22,573

1. Azman Noh (BN-UMNO)

2. Abul Jais Ashfaq Ahmed (PH-PKR)

3. Ahmad Faisal Mansor (PN)

4. Halim (PEJUANG)

PRU14:DATUK ABD MANAF (BN)

Jumlah Undi: 6,312

Majoriti: 1,624

N37 PANTAI REMIS

Jumlah Pengundi: 44,432

1. Looi Tuan Gin (BN-MCA)

2. Ee Chin Oon (PN-GERAKAN)

3. Jason Ng Thien Yeong (PH)

PRU14: WONG MAY ING (PKR)

Jumlah Undi: 18,608

Majoriti: 13,316

N38 ASTAKA

Jumlah Pengundi: 41,244

1. Chee Lee Chong (BN-MCA)

2. Yong Il Yan (PN-GERAKAN)

3. Jason Ng Thien Yeong (PH)

PRU14:Teoh Yee Chern (PKR)

Jumlah Undi: 18,814

Majoriti: 15,176

N39 BELANJA

Jumlah Pengundi: 17,767

1. Khairudin Abu Hanipah (BN-UMNO)

2. Wan Meor Safwat Naqiuddin Shamsudin (PN-BERSATU)

3. Shaharuzzman (PEJUANG)

4. Hishamuddin Abdullah (PH)

PRU14:KHAIRUDIN ABU HANIPAH (BN)

Jumlah Undi: 5,879

Majoriti: 2,729

N40 BOTA

Jumlah Pengundi: 30,148

1. Khairul Shahril Mohamed (BN)

2. Dr Najihatussaleh Ahmad (PN-PAS)

3. Usaili Alias (PH-PKR)

4. Shahril Harabab (GTA)

PRU14:KHAIRUL SHAHRIL MOHAMED (BN)

Jumlah Undi: 7,411

Majoriti: 1,822

P70 KAMPAR

Jumlah Pengundi: 89,894

1. Lee Chee Leong (BN-MCA)

2. Wong Oi Foon (PN-GERAKAN)

3. Chong Zhemin (PH)

4. Leong Cheok Lung (Warisan)

PRU14: SU KEONG SIONG (PKR)

Jumlah Undi: 30,216

Majoriti : 11,801

N41 MALIM NAWAR

Jumlah Pengundi: 29,701

1. Chin Woon Kheong (BN-MCA)

2. Sherry Syed (PN)

3. Leong Cheok Keng (Warisan)

4. Shasha (PH)

PRU14:LEONG CHEOK KENG (PKR)

Jumlah Undi: 11,271

Majoriti: 6,082

N42 KERANJI

Jumlah Pengundi: 28,248

1. Ng Wah Leng (BN-MCA)

2. Foong Kar Sing (PN-GERAKAN)

3. Koo Haai Yen (PH)

4. Puah Chee Haur (Warisan)

PRU14:CHONG ZHEMIN (PKR)

Jumlah Undi: 12,072

Majoriti: 8,041

N43 TUALANG SEKAH

Jumlah Pengundi: 31,945

1. Abd Rahman Md Som (BN-UMNO)

2. Fazli Abdullah Sani (PN-BERSATU)

3. Mohd Azlan Helmi Helmi (PH-PKR)

PRU14:NOLEE RADZI (BN)

Jumlah Undi: 8,767

Majoriti: 1,099

N44 SUNGAI RAPAT

Jumlah Pengundi: 47,931

1. Hang Tuah Din (BN-UMNO)

2. Roshanita Basir (Warisan)

3. Mohader Ahmad Mohammad Ayob (PN-BERSATU)

4. Mohammad Nizar Jamaluddin (PH)

PRU14:DATUK MOHAMMAD NIZAR JAMALUDDIN (PKR)

Jumlah Undi: 12,425

Majoriti: 3,614

N45 SIMPANG PULAI

Jumlah Pengundi: 62,756

1. Lee Wai Yin (BN-MCA)

2. Selvam Kunjambu (PN-BERSATU)

3. Wong Chai Yi (PH-PKR)

4. Hooi Mi Suet (Warisan)

PRU14:TAN KAR HING (PKR)

Jumlah Undi: 25,659

Majoriti: 20,689

N46 TEJA

Jumlah Pengundi: 32,970

1. Liew Yee Lin (BN-MCA)

2. Low Leong Sin (Warisan)

3. Ahmad Ishak (PN-BERSATU)

4. Sandrea Ng Shy Ching (PH-PKR)

PRU14:SANDREA NG (PKR)

Jumlah Undi: 10,546

Majoriti: 4,390

N47 CHENDERIANG

Jumlah Pengundi: 30,006

1. Choong Shin Heng (BN-MCA)

2. Ahmad Tarmizi Ghazali (Warisan)

3. Mohd Yunus Mohd Yusof (PN-BERSATU)

4. Atyrah Hanim Razali (PH-PKR)

PRU14:Ahmad Faizal Azumu (PKR)

Jumlah Undi: 7,662

Majoriti: 39

N48 AYER KUNING

Jumlah Pengundi: 31,940

1. Muhammad Noor Farid Zainal (PN-PAS)

2. Noor Hasnida Hashim (Warisan)

3. Isham Shahruddin (BN-UMNO)

PRU14: DATUK SAMSUDIN ABU HASSAN (BN)

Jumlah Undi: 9,141

Majoriti: 2,087

N49 SUNGAI MANIK

Jumlah Pengundi: 35,520

1. Zainol Fadzi Paharudin (PN-BERSATU)

2. Mohamad Maharani Md Tasi (PH-PKR)

3. Ibrahim Katop (BN)

PRU14:DATUK ZAINOL FADZI PAHARUDIN (BN)

Jumlah Undi: 9,265

Majoriti: 1,939

N50 KAMPONG GAJAH

Jumlah Pengundi: 39,241

1. Wan Norashikin Wan Noordin (BN-UMNO)

2. Zafarulazlan Zan (PN-PAS)

3. Mohd Syamsul Bin Alauddin (PH)

4. Mohd Safian Sauri Pandak Abd Samad (PEJUANG)

PRU14:DATUK DR WAN NORASHIKIN WAN NOORDIN (BN)

Jumlah Undi: 11,026

Majoriti: 3,345

P74 LUMUT

Jumlah Pengundi: 92,972

1. Nordin Ahmad Ismail (PN-BERSATU)

2. Mohd Hatta Ramli (PH-AMANAH)

3. Isnin Ismail @ Ibrahim Khan (Warisan)

4. Mazlan Abdul Ghani (PEJUANG)

5. Zambry Abd Kadir (BN-UMNO)

PRU14: DR HATTA RAMLI (PKR)

Jumlah Undi: 21,955

Majoriti : 400

N51 PASIR PANJANG

Jumlah Pengundi: 62,418

1. Mohd Rafiq Mohd (BN-UMNO)

2. Nur Inderasyawalis Ahmad Mukhtar (Warisan)

3. Rosli Abd Rahman (PN-PAS)

4. Yahaya (PH)

5. Mohd Norazlee Mohd (GTA)

PRU14: YAHAYA MD NOr (PKR)

Jumlah Undi: 14,123

Majoriti: 1,219

N52 PANGKOR

Jumlah Pengundi: 30,554

1. Norzaman Man (BN-UMNO)

2. Yahaya Ahmad (Warisan)

3. Norhaslinda Zakaria (PN-BERSATU)

4. Abdul Razak Nazri (PH-PKR)

5. Mohd Azlan Suhaimi (GTA)

PRU14: DATUK SERI DR ZAMBRY ABD KADIR

Jumlah Undi: 8,378

Majoriti: 1,626

N53 RUNGKUP

Jumlah Pengundi: 19,670

1. Shahrul Zaman Yahaya (BN-UMNO)

2. Mohd Mokheri Jalil (PN-PAS)

3. Munzirie Kabir (PH)

PRU14: SHAHRUL ZAMAN YAHYA (BN)

Jumlah Undi: 6,529

Majoriti: 3,069

N54 HUTAN MELINTANG

Jumlah Pengundi: 38,513

1. Khairuddin Tarmizi (BN-UMNO)

2. Khairun Nizam Marsom (PN-BERSATU)

3. S Wasanthee (PH)

PRU14: KHAIRUDDIN TARMIZI (BN)

Jumlah Undi: 10,961

Majoriti: 741

N55 PASIR BEDAMAR

Jumlah Pengundi: 39,117

1. Suriyanarayanan Sannasy Naidu (PN-PAS)

2. Kong Sun Chin (BN-MCA)

3. Woo Kah Leong (PH)

PRU14:TERENCE NAIDU (PKR)

Jumlah Undi: 19,480

Majoriti: 14,520

N56 CHANGKAT JONG

Jumlah Pengundi: 48,105

1. Mohd Azhar Jamaluddin (BN-UMNO)

2. Nadziruddin Mohamed Bandi (PN-BERSATU)

3. Badrul Hisham Badarudin (PH)

4. Muhammad Amirul Mahfuz (PEJUANG)

PRU14:MOHD AZHAR JAMALUDDIN (BN)

Jumlah Undi: 11,216

Majoriti: 67

N57 SUNGKAI

Jumlah Pengundi: 23,375

1. A Sivanesan (PH)

2. Thilak Raj Gunasekaran (PN-PAS)

3. Elango (BN)

4. Mahhadee (Bebas)

PRU14:A SIVANESAN (PKR)

Jumlah Undi: 9,631

Majoriti: 6,493

N58 SLIM

Jumlah Pengundi: 30,581

1. Mohd Zaidi Aziz (BN-UMNO)

2. Muhammad Zulfadli Zainal (PN-PAS)

3. Mohd Syahid Mohd Zainu (PKR-PH)

PRU14:DATUK KHUSAIRI TALIB (BN)

Jumlah Undi: 8,327

Majoriti: 2,183

N59 BEHRANG

Jumlah Pengundi: 39,917

1. Salina Samsudin (BN-UMNO)

2. Mohd Amran Ibrahim (PN-BERSATU)

3. Hazvee Hafiz (PEJUANG)

4. Khairol Najib (PH)

PRU14: AMINUDDIN ZULKIPLI (PKR)

Jumlah Undi: 9,770

Majoriti: 409

PAHANG

N1 TANAH RATA

Jumlah Pengundi: 26,838

1. Ho Chi Yang (PH-DAP)

2. Wong Yap Wah (BN-MCA)

3. Lai Chii Wen (PN-GERAKAN)

PRU14:CHIONG YOKE KONG (PKR)

Jumlah Undi: 8,821

Majoriti: 3,589

N2 JELAI

Jumlah Pengundi: 19,182

1. Ismail Mohd Hussin (PH-PKR)

2. Wan Rosdy Wan Ismail (BN-UMNO)

3. Abdul Rasid Mohamed Ali (PN-BERSATU)

PRU14:DATUK SERI WAN ROSDY WAN ISMAIL (BN)

Jumlah Undi: 5,858

Majoriti: 3,507

N3 PADANG TENGKU

Jumlah Pengundi: 19,409

1. Ruzi @ Nata Yusuff (PH-AMANAH)

2. Mustapa Long (BN-UMNO)

3. Mohd Rostam Mustapha (PEJUANG)

4. Muhamad Khaider Kamil (PN-PAS)

PRU14: DATUK MUSTAPA LONG (BN)

Jumlah Undi: 5,691

Majoriti: 1,244

N4 CHEKA

Jumlah Pengundi: 16,822

1. Datuk Seri Tuan Ibrahim Tuan Man (PN-PAS)

2. Dr Rasid Muhamad (PH-PKR)

3. Aishaton Abu Bakar (PEJUANG)

4. Ho Fong Mee (BN-MCA)

PRU14: LEE AH WONG (BN)

Jumlah Undi: 3,475

Majoriti: 202

N5 BENTA

Jumlah Pengundi: 10, 895

1. Rizal Jamin (PH-PKR)

2. Mohd Soffi Abdul Razak (BN-UMNO)

3. Nor Alias Ibrahim (PN-Bersatu)

PRU14: DATUK SERI MOHD SOFFI ABD RAZAK (BN)

Jumlah Undi: 3,261

Majoriti: 340

N6 BATU TALAM

Jumlah Pengundi: 19,683

1. Datuk Abd Aziz Mat Kiram (BN-UMNO)

2. Shahuddin Abdul Rahman (PH-PKR)

3. Ahmad Sabri Mat Dui (PN-BERSATU)

PRU14: ABD AZIZ MAT KIRAM (BN)

Jumlah Undi: 6,922

Majoriti: 4,320

N7 TRAS

Jumlah Pengundi: 38,329

1. Tengku Zulpuri Shah Raja Puji (PH-DAP)

2. Amirul Mukminin Kuek Abdullah (PN-GERAKAN)

3. Lim Teck Hoe (BN-MCA)

4. Mohd Tahir Kassim (PEJUANG)

PRU14: CHOW YU HUI (PKR)

Jumlah Undi: 15,660

Majoriti: 9,953

N8 DONG

Jumlah Pengundi: 17,052

1. Fadzli Mohamad Kamal (BN-UMNO)

2. Mohd Abd Jawaad Abd Ghafar @ Razmin (PH-AMANAH)

3. Tengku Shah Amir Tengku Perang (PN-PAS)

PRU14: SHAHIRUDDIN AB MOIN (BN)

Jumlah Undi: 6,294

Majoriti: 3,832

N9 TAHAN

Jumlah Pengundi: 20,895

1. Mohd Abd Talib Mohd Tahar (PH-AMANAH)

2. Faezah Ishak (BN-UMNO)

3. Mohd Zakhwan Ahmad Badarddin (PN-PAS)

PRU14: MOHD ZAKHWAN AHMAD BADARDDIN (PAS)

Jumlah Undi: 5,638

Majoriti: 69

N10 DAMAK

Jumlah Pengundi: 34,967

1. Muhamad Fasal Jamlus (BN-UMNO)

2. Mohamad Rafly Mohd Satar (PH-PKR)

3. Izzuddin Ismail (BEBAS)

4. Zuridan Mohd Daud (PN-PAS)

PRU14: ZURIDAN MD DAUD (PAS)

Jumlah Undi: 8,444

Majoriti: 1,456

N11 PULAU TAWAR

Jumlah Pengundi: 31,189

1. Norani Muhd @ Mohd Arshad (PH-AMANAH)

2. Mohd Rosidi Hassan (PEJUANG)

3. Ridzuan Mansor (BEBAS)

3. Yohanis Ahmad (PN-PAS)

4. Nazri Ngah (BN-UMNO)

PRU14: DATUK NAZRI NGAH (BN)

Jumlah Undi: 8,446

Majoriti: 587

N12 BESERAH

Jumlah Pengundi: 59,370

1. Zulkifli Mohamed (PH-AMANAH)

2. Joseph Tang (BEBAS)

3. Mustaffar Kamal Ab Hamid (BN-UMNO)

4. Andansura Rabu (PN-PAS)

5. Mohd Pauzi Taib (PEJUANG)

PRU14: ANDANSURA RABU (PAS)

Jumlah Undi: 12,239

Majoriti: 1,305

N13 SEMAMBU

Jumlah Pengundi: 61,179

1. Chan Chun Kuang (PH-PKR)

2. Khairul Hisham Omar (BN-UMNO)

3. Zulfadhli Zakariah (PN-PAS)

4. Dr Rashidah Abdul Rahman (PEJUANG)

PRU14: LEE CHEAN CHUNG (PKR)

Jumlah Undi: 14,991

Majoriti: 5,511

N14 TERUNTUM

Jumlah Pengundi: 30,854

1. Tee Choon Ser (BN-MCA)

2. Yap Kim Heng (PN-GERAKAN)

3. Khairul Afifie Abdullah (PEJUANG)

4. Sim Chon Siang (PH-PKR)

PRU14: SIM CHON SIANG (PKR)

Jumlah Undi: 12,227

Majoriti: 7,725

N15 TANJUNG LUMPUR

Jumlah Pengundi: 38,185

1. Rosli Abdul Jabar (PN-PAS)

2. Nara@Nikman Nordin (BN-UMNO)

3. Ab 'alim Ruslam Ahmad (PEJUANG)

4. Sabrina Md Yusoff (PH-Amanah)

PRU14:ROSLI ABDUL JABAR (PAS)

Jumlah Undi: 8,198

Majoriti: 1,339

N16 INDERAPURA

Jumlah Pengundi: 18,558

1. Norashikin Ismail (PEJUANG)

2. Shafik Fauzan Sharif (BN-UMNO)

3. Fakhrul Anuar Zulkawi (PH-PKR)

4. Mohd Hafizal Mohamed Shah (PN-PAS)

PRU14:DATUK SERI SHAFIK FAUZAN SHARIF (BN)

Jumlah Undi: 5,568

Majoriti: 2,165

N17 PAYA LEMBING

Jumlah Pengundi: 19,130

1. Anuar Tajuddin (PEJUANG)

2. Umar Arbain Dollah (BN-UMNO)

3. Mohamad Ayub Asri (PN-PAS)

4. Ahmad Omar (PH-PKR)

PRU14: DATUK MD SOHAIMI MOHAMED SHAH (BN)

Jumlah Undi: 5,436

Majoriti: 1,968

N18 LEPAR

Jumlah Pengundi: 25,728

1. Mohd Yazid Mohd Yunus (PN-BERSATU)

2. Rosminahar Mohd Amin (PEJUANG)

3. Muhammad Ibrohim Mazalan (PH-AMANAH)

4. Abd Rahim Muda (BN-UMNO)

PRU14: ABD RAHIM MUDA (BN)

Jumlah Undi: 6,500

Majoriti: 2,779

N19 PANCHING

Jumlah Pengundi: 34,896

1. Fauziah Abd Wahab (BN-UMNO)

2. Mohd Tarmizi Yahya (PN-PAS)

3. Haslindalina Hashim (PH-PKR)

4. Afif Syairol Abdul Rahim (PEJUANG)

PRU14: MOHD TARMIZI YAHAYA (PAS)

Jumlah Undi: 7,143

Majoriti: 135

N20 PULAU MANIS

Jumlah Pengundi: 30,113

1. Khairuddin Mahmud (BN-UMNO)

3. Muhammad Khairil Khalid (PH-DAP)

2. Mohammad Rafiq Khan Ahmad Khan (PN-PAS)

PRU14:DATUK KHAIRUDDIN MAHMUD (BN)

Jumlah Undi: 8,282

Majoriti: 2,334

N21 PERAMU JAYA

Jumlah Pengundi: 41,915

1. Tugimon Abd Hamid (PH-AMANAH)

2. Tengku Zainul Hisham Tengku Hussin (BEBAS)

3. Mohammad Nizar Mohamad Najib (BN-UMNO)

4. Abu Talib Muhammad (PN-BERSATU)

PRU14: SH MOHMED PUZI (BN)

Jumlah Undi: 15,433

Majoriti: 7,182

N22 BEBAR

Jumlah Pengundi: 21,294

1. Ibrahim Sulaiman (PH-AMANAH)

2. Narzatul Haidar Sakim (PN-BERSATU)

3. Mohammad Fakhruddin Mohd Ariff (BN-UMNO)

PRU14:MOHAMMAD FAKHRUDDIN MOHD ARIFF (BN)

Jumlah Undi: 8,172

Majoriti: 5,720

N23 CHINI

Jumlah Pengundi: 26,121

1. Mohd Sharim Md Zain (BN-UMNO)

2. Mohamad Yazid Che Mat (PH-PKR)

3. Fahrur Rozi Jalaluddin (PN-PAS)

PRU14:DATUK SERI ABU BAKAR HARUN (BN)

Jumlah Undi: 10,027

Majoriti: 4,622

N24 LUIT

Jumlah Pengundi: 16,279

1. Jamaluddin Md Marzuki (BN-UMNO)

2. Mohd Sofian Abd Jalil (PN-PAS)

3. Mohd Afandi Abd Rani (PH-PKR)

PRU14:MOHD SOFIAN ABD JALIL (PAS)

Jumlah Undi: 4,238

Majoriti: 272

N25 KUALA SENTUL

Jumlah Pengundi: 17,434

1. Jasri Jamaludin (PN-BERSATU)

2. Mohd Khairuddin Mohd Ali Hanafiah (BN-UMNO)

3. Amran Tahir (PH-PKR)

PRU14: SHAHANIZA SHAMSUDDIN (BN)

Jumlah Undi: 5,541

Majoriti: 2,039

N26 CHENOR

Jumlah Pengundi: 19,415

1. Nor Hisham Mohd Suki (PH-Amanah)

2. Saiful Anuar Mokhtar (BN-UMNO)

3. Mujibur Rahman Ishak (PN-PAS)

PRU14: MUJIBUR RAHMAN ISHAK (PAS)

Jumlah Undi: 6,016

Majoriti: 514

N27 JENDERAK

Jumlah Pengundi: 13,994

1. Mohamad Hafez Harun (PH-PKR)

2. Md Anuar Daud (PN-PAS)

3. Rodzuan Zaaba (BN-UMNO)

PRU14:DATUK SERI MOHAMED JAAFAR (BN)

Jumlah Undi: 4,213

Majoriti: 1,713

N28 KERDAU

Jumlah Pengundi: 15,184

1. Mohd Arhan Mohd Saludin (PN-BERSATU)

2. Ahmad Ariff Md Daud (PH-AMANAH)

3. Syed Ibrahim Syed Ahmad (BN-UMNO)

PRU14:SYED IBRAHIM SYED AHMAD (BN)

Jumlah Undi: 5,255

Majoriti: 1,934

N29 JENGKA

Jumlah Pengundi: 31,359

1. Norhasmimi Abdul Ghani (BN-UMNO)

2. Shahril Azman Abdul Halim (PN-PAS)

3. Jamaluddin Abd Rahim (PH-AMANAH)

PRU14:SHAHRIL AZMAN ABD HALIM AL HAFIZ (PAS)

Jumlah Undi: 10,111

Majoriti: 1,813

N30 MENTAKAB

Jumlah Pengundi: 37,269

1. Abdul Wahab Kadir (PEJUANG-PUTRA)

2. Wong Tze Shiang (BN-MCA)

3. Mohd Yusoff Abdullah (PN-BERSATU)

4. Woo Chee Wan (PH-DAP)

PRU14: WOO CHEE WAN (PKR)

Jumlah Undi: 10,547

Majoriti: 5,829

N31 LANCHANG

Jumlah Pengundi: 32,774

1. Abd Jamil Yaakub (PRM)

2. Razmi Sudin (BN-UMNO)

3. Hassan Omar (PN-PAS)

4. Zaini Mohamad (PEJUANG)

5. Rosli Ismail @ Ahmad(PH-PKR)

PRU14:DATUK SERI MOHD SHARKAR SHAMSUDIN (BN)

Jumlah Undi: 8,824

Majoriti: 3,725

N32 KUALA SEMANTAN

Jumlah Pengundi: 36,786

1. Mohd Syafiq Mohd Khaidir (PEJUANG)

2. Hassanuddin Salim (PN-PAS)

3. Datuk Nor Azmi Mat Ludin (BN-UMNO)

4. Azizul Shah Mohd Noor (PH-AMANAH)

PRU14:DATUK NOR AZMI MAT LUDIN (BN)

Jumlah Undi: 7,880

Majoriti: 474

N33 BILUT

Jumlah Pengundi: 24,286

1. Mohd Shokri Mahmood (PEJUANG-PUTRA)

2. B. Sri Chandra (PN-Gerakan)

3. Wong Siew Mun (BN-MCA)

4. Lee Chin Chen (PH-DAP)

PRU14: LEE CHIN CHEN (PKR)

Jumlah Undi: 8,798

Majoriti: 3,739

N34 KETARI

Jumlah Pengundi: 30, 758

1. Thomas Su Keong Siong (PH-DAP)

2. Tan Wei Leong (PN-GERAKAN)

3. Datuk Amizar Abu Adam (BN-UMNO)

PRU14:YOUNG SYEFURA OTHMAN (PKR)

Jumlah Undi: 9,873

Majoriti: 3,710

N35 SABAI

Jumlah Pengundi: 15,643

1. Nurul Qomar Abdol Talib @ Ramali (PN-Bersatu)

2. D. Kamache (PH-DAP)

3. V. Arumugam (BN-MIC)

PRU14:D KAMACHE (PKR)

Jumlah Undi: 4,374

Majoriti: 495

N36 PELANGAI

Jumlah Pengundi: 16,371

1. Johari Harun (BN-UMNO)

2. Isa Ahmad (PEJUANG-PUTRA)

3. Ahmed Wafiuddin Shamsuri (PH-AMANAH)

4. Kasim Samat (PN-PAS)

PRU14: DATUK SERI ADNAN YAAKOB (BN)

Jumlah Undi: 5,410

Majoriti: 2,312

N37 GUAI

Jumlah Pengundi: 24,412

1. Jafari Mohd Yusof (BEBAS)

2. Nor Hashimah Mat Noh (PN-BERSATU)

3. Datuk Sabariah Saidan (BN-UMNO)

4. Noraini Abdul Ghani (PH-AMANAH)

PRU14: NOROL AZALI SULAIMAN (BN)

Jumlah Undi: 8,147

Majoriti: 3,295

N38 TRIANG

Jumlah Pengundi: 25,981

1. Muhammad Izzuddin Zulkifli (PN-BERSATU)

2. Yee Cheng Hwa (BN-MCA)

3. Leong Yu Man (PH-DAP)

PRU14:LEONG YU MAN (PKR)

Jumlah Undi: 11,222

Majoriti: 6,454

N39 KEMAYAN

Jumlah Pengundi: 27,366

1. Shapuan Hussin (PN-PAS)

2. Khaizulnizam Mohamad Zuldin (BN-UMNO)

PRU14: MOHD FADIL OSMAN (BN)

Jumlah Undi: 8,430

Majoriti: 4,618

N40 BUKIT IBAM

Jumlah Pengundi: 31,003

1. Nazri Ahmad (PN-PAS)

2. Barirah Mohd Mokhtar (PH-PKR)

3. Datin Samsiah Arshad (BN-UMNO)

PRU14: SAMSIAH ARSHAD (BN)

Jumlah Undi: 9,448

Majoriti: 2,478

N41 MUADZAM SHAH

Jumlah Pengundi: 29,663

1. Jaafar Mustaffa (PN-Bersatu)

2. Razali Kassim (BN-UMNO)

3. Norlaily Forizad (PH-PKR)

PRU14: RAZALI KASSIM (BN)

Jumlah Undi: 9,081

Majoriti: 4,840

N42 TIOMAN

Jumlah Pengundi: 28,465

1. Sulaiman Bakar (BEBAS)

2. Osman A. Bakar (PEJUANG)

3. Md Yunus Ramli (PN-PAS)

4. Mohd Fadzli Mohd Ramly (PH-AMANAH)

5. Datuk Seri Mohd Johari Hussain (BN-UMNO)

PRU14: MOHD JOHARI HUSSAIN (BN)

Jumlah Undi: 6,662

Majoriti: 1,280

"""

In [40]:
split_list = name_list.split("\b")
for i in split_list:
    print(i)


N01 Titi Tinggi

1. Teh Chai Aan (BN)

2. Teh Seng Chuan (PH)

3. Izizam Ibrahim (PN)

4. Khaw Hock Kong (WARISAN)

5. Mohd Shahril Md Sarif (PEJUANG)

6. Zahidi Zainul Abidin (Bebas)

Jumlah pengundi PRU15: 10,111

PRU14 Penyandang: Teh Chai Aan (BN)

Majoriti: 142

N02 Beseri

1. Ruzaini Rais (BN)

2. Mat Safar Saad (PH)

3. Haziq Asyraf Dun (PN)

4. Mohd Shamim M Nordin (WARISAN)

Jumlah pengundi PRU15: 8,914

PRU14: Penyandang: Ruzaini Rais (BN)

Majoriti: 416

N03 Chuping

1. Asmaiza Ahmad (BN)

2. Natthavuth Seng (PH)

3. Saad Seman (PN)

Jumlah pengundi PRU15: 10,542

PRU14 Penyandang: Asmaiza Ahmad (BN)

Majoriti: 1,367

N04 Mata Ayer

1. Wan Badariyah Wan Saad (PN)

2. Amran Kamarudin (PH)

3. Siti Berenee Yahya (BN)

Jumlah Pengundi PRU15: 7,617

PRU14: Penyandang: Siti Berenee Yahya (BN)

Majoriti: 720

N05 Santan

1. Che Mazlina Che Yob (PH)

2. Azmir Azizan (PN)

3. Azizan Sulaiman (BN)

Jumlah penngundi PRU15: 8,912

PRU14: Penyandang Azizan Sulaiman (BN)

Majoriti: 949
